In [27]:
# # reading harvardx data from the csv
# # still a work in progress since their dataset is malformatted
# import csv

# # load data
# csvfile_path = '../datasets/corpus_HarvardX_LatestCourses_based_on_2016-10-18.csv'

# with open(csvfile_path, 'r') as csvfile:
#     reader = csv.DictReader(csvfile)
#     print reader.next()

In [28]:
# reading udacity stuff
import json
from whoosh.fields import *

# load data 
udacity_path = 'datasets/udacity-api.json'

udacity_data = None
with open(udacity_path, 'r') as file:
    udacity_data = json.load(file)
    
# print udacity_data['courses']


# set up whoosh
# schema 
schema = Schema(
    slug=ID(stored=True),
    title=TEXT(stored=True),
    subtitle=TEXT,
    expected_learning=TEXT,
    syllabus=TEXT,
    summary=TEXT,
    short_summary=TEXT
)

print schema

<Schema: ['expected_learning', 'short_summary', 'slug', 'subtitle', 'summary', 'syllabus', 'title']>


In [29]:
# make an index
import os.path
from whoosh.index import create_in, open_dir

# make an index to store this stuff in 
index_path = 'whoosh_indices'
if not os.path.exists(index_path):
    os.mkdir(index_path)
index = create_in(index_path, schema)

print index

FileIndex(FileStorage('whoosh_indices'), 'MAIN')


In [63]:
# reopen index
from whoosh.index import open_dir

index = open_dir(index_path)


In [78]:
# start adding documents (i.e. the courses) to the index 

try:
    writer = index.writer()
    for course in udacity_data['courses']:
#         print "Writing course " + course['title']
        writer.add_document(
            slug=course['slug'],
            title=course['title'],
            subtitle=course['subtitle'],
            expected_learning=course['expected_learning'],
            syllabus=course['syllabus'],
            summary=course['summary'],
            short_summary=course['short_summary'])
    writer.commit()
except Exception as e:
#     print ":("
    print e

In [85]:
# search
from whoosh.query import *
from whoosh.qparser import MultifieldParser

# query parser so we can enter normal english language queries
fields = ["title", "subtitle", "expected_learning", "syllabus", "summary", "short_summary"]
parser = MultifieldParser(fields, index.schema)

with index.searcher() as searcher:
    # run a test search
    myquery = parser.parse("computer vision")
    results = searcher.search(myquery)
    print results
    
    

<Top 10 Results for And([Or([Term('title', u'computer'), Term('subtitle', u'computer'), Term('expected_learning', u'computer'), Term('syllabus', u'computer'), Term('summary', u'computer'), Term('short_summary', u'computer')]), Or([Term('title', u'vision'), Term('subtitle', u'vision'), Term('expected_learning', u'vision'), Term('syllabus', u'vision'), Term('summary', u'vision'), Term('short_summary', u'vision')])]) runtime=0.0512809753418>
